# What is a Product Recommender system?
1. Product recommendation systems work at the core of ecommerce industry. Customers use it for browsing through millions of products so they can buy their favourite items. on the other hand ecommerce shop owners/creators use them to study market and build and stock their inventory. Its not limited to ecommerce, recommender systems find applications in Entertainment domain where ott platforms suggest users content based on what they previously watched.

2. There are six types of Recommendation System
  1. Collaborative 
  2. Content based 
  3. Demography based 
  4. Utility based 
  5. Knowledge based 
  6. Hybrid 

    




# After reading the problem statement one point is clear. Here the creator needs content based recommendation system.

They are providing a video/pebble of each item alongwith products that are used to create that video. This is an implicit type of content sharing.
in solving this there are two approaches If creator wants to take into account the like counts, view count its a numerical data type so euclidean distance has to be calculated.

If creator takes into consideration type of content like its media format then it will be a text based data. to analyse text based features we have to calculate Cosine similarity. there is another type called Jaccard similarity used for categorical data.

In my opinion Machine learning will be helpful for creator to build the shop and make it more interactive. relevant content suggestion for uploading video of an item is a deciding factor here. if creator want to omit ML approaches they have to build and maintain a dymanic website that needs update every time a new event occurs like adding new items for categories.

# Import libraries 

In [174]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [175]:
import json

In [176]:
with open('content_masked.json','r') as f:
    content=json.loads(f.read())

In [177]:
content_final=pd.json_normalize(content)

In [178]:
content_final.head(10)

,type,media_type,influencer_ids,brand_ids,is_processed,is_active,view_count,like_count,comment_count,caption,...,series_ids,_ids.$oid,label.interests,label.age_groups,label.genders,created_at.$date,processed_at.$date,_id.$oid,last_sync.$date,label
0,pebble,video,[{'$oid': '*****************4055'}],[{'$oid': '*****************be8b'}],True,False,0,0,0,Test pebble,...,None,*****************b634,[interest],"[26-30, 30-35]","[M, F]",2021-04-19T09:47:58.735Z,2021-04-19T09:48:42.575Z,NaN,NaN,NaN
1,pebble,video,[{'$oid': '*****************033e'}],[{'$oid': '*****************fe62'}],True,True,206,9,0,A curated box of surprise samples from select ...,...,None,NaN,"[Sample, testers, free products]","[26-30, 30-35]","[M, F]",2021-04-19T11:59:46.297Z,2021-04-19T12:00:32.067Z,*****************b698,2022-02-12T09:34:43.513Z,NaN
2,pebble,video,[{'$oid': '*****************f57a'}],[{'$oid': '*****************fb82'}],True,True,501,10,0,Ways to Wear Black Cotton Trousers\n\nThe tape...,...,None,NaN,"[work wear, work, women, office look, professi...","[26-30, 30-35]",[F],2021-04-19T19:02:53.362Z,2021-04-19T19:04:14.088Z,*****************b6dc,2022-02-08T08:34:19.771Z,NaN
3,pebble,video,[{'$oid': '*****************f57b'}],[{'$oid': '*****************93d3'}],True,True,16626,1471,0,She's anything but ordinary. Wearing Kica Acti...,...,[{'$oid': '*****************417e'}],NaN,"[work out, activewear, gym, workout, body toni...","[26-30, 30-35]",[F],2021-04-19T19:18:29.974Z,2021-04-19T19:19:23.685Z,*****************b6fa,2022-02-14T15:51:44.728Z,NaN
4,pebble,video,[{'$oid': '*****************f57d'}],[{'$oid': '*****************9591'}],True,True,263,4,0,Midweek feels. There's so much more to a week ...,...,None,NaN,"[shirt, graphic shirt, quirky, orinted shirt, ...","[26-30, 30-35]",[M],2021-04-19T19:28:25.102Z,2021-04-19T19:29:17.259Z,*****************b6fe,2022-02-15T05:35:58.946Z,NaN
5,pebble,video,[{'$oid': '*****************f580'}],[{'$oid': '*****************4533'}],True,True,2252,104,0,"Vibrant Tropical Shirt from 15 buttons, styled...",...,"[{'$oid': '*****************40f5'}, {'$oid': '...",NaN,"[tailored , custom, printed shirts, casual, ev...","[26-30, 30-35]",[M],2021-04-19T19:54:46.075Z,2021-04-19T19:55:42.846Z,*****************b714,2022-02-15T05:33:31.666Z,NaN
6,pebble,video,[{'$oid': '*****************f581'}],[{'$oid': '*****************1058'}],True,True,385,3,0,Experience absolute beast mode during workouts...,...,None,NaN,"[work out, gym, supplements, protien, energy, ...","[26-30, 30-35]",[M],2021-04-19T20:05:36.859Z,2021-04-19T20:06:30.074Z,*****************b719,2022-01-26T14:47:40.379Z,NaN
7,pebble,video,[{'$oid': '*****************033e'}],[{'$oid': '*****************1058'}],True,True,149,4,0,Keeps you active and energetic throughout the ...,...,None,NaN,"[workout, gym, vitamin, energy, protien, hydra...","[26-30, 30-35]","[M, F]",2021-04-21T21:35:05.719Z,2021-04-21T21:35:49.683Z,*****************bbdc,2022-02-12T09:34:43.513Z,NaN
8,pebble,video,[{'$oid': '*****************f5c3'}],[{'$oid': '*****************11a9'}],True,True,69,1,0,Keep your skin hydrated with @prolixr All-New ...,...,None,NaN,"[skincare, beauty, makeup, looking good, routi...","[26-30, 30-35]",[F],2021-04-21T21:41:51.915Z,2021-04-21T21:42:25.552Z,*****************bbe0,2022-02-15T05:38:50.576Z,NaN
9,pebble,video,[{'$oid': '*****************f5c4'}],[{'$oid': '*****************93d3'}],True,True,15516,1422,0,"A hypnotising flow, and nope we aren’t going t...",...,[{'$oid': '*****************4178'}],NaN,"[kica active, kica, fitness, gym wear, active ...","[26-30, 30-35]",[F],2021-04-21T21:48:49.687Z,2021-04-21T21:50:14.700Z,*****************bbe4,2022-02-14T15:51:44.560Z,NaN


In [179]:
content_final.columns

Index(['type', 'media_type', 'influencer_ids', 'brand_ids', 'is_processed',
       'is_active', 'view_count', 'like_count', 'comment_count', 'caption',
       'catalog_ids', 'catalog_info', 'like_ids', 'liked_by', 'last_sync',
       'category_path', 'hashtags', 'sync_type', 'sync', 'series_ids',
       '_ids.$oid', 'label.interests', 'label.age_groups', 'label.genders',
       'created_at.$date', 'processed_at.$date', '_id.$oid', 'last_sync.$date',
       'label'],
      dtype='object')

In [180]:
features=['type','media_type','influencer_ids','brand_ids','caption','label.interests']

In [181]:
for feature in features:
    content_final[feature] = content_final[feature].fillna('')

In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [183]:
from sklearn.metrics.pairwise import linear_kernel

In [184]:
tf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tf.fit_transform(content_final['caption'])


In [185]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [186]:
indices = pd.Series(content_final.index, index=content_final['type']).drop_duplicates()

In [187]:
def get_recommend(type,cosine_sim=cosine_sim):
    idx=indices[type]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:5]
    get_index=[i[0] for i in sim_scores]
    
    return content_final['type'].iloc[get_index]



    


In [188]:
get_recommend(11)

51      pebble
66      pebble
1885    pebble
368     pebble
Name: type, dtype: object

In [189]:
get_recommend(4)

4973    pebble
34      pebble
7465    pebble
2597    pebble
Name: type, dtype: object

# the end

Do you think the given data points are sufficient for building the product recommendation engines for the creators? 

Even though these data points can give predictions but they lack a significant feature like rating of products. like and view counts can only predict popularity of the item but rating of a product makes it reliable from user's POV and helps content creator to increase stock of most reviewed product to improve sales.